In [2]:
import os
import sys
import json
import time
import datetime
from importlib import reload

import requests

import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

import external as ext

In [3]:
desert_main = pd.read_csv('miramar.csv')
baltic_main = pd.read_csv('erangel.csv')
savage_main = pd.read_csv('sanhok.csv')
summerland_main = pd.read_csv('karakin.csv')

In [4]:
desert_main = desert_main.iloc[:,1:]
baltic_main = baltic_main.iloc[:,1:]
savage_main = savage_main.iloc[:,1:]
summerland_main = summerland_main.iloc[:,1:]

In [9]:
def risklevel(x,y,mapname):
    if mapname == 'Desert_Main':
        try:
            risk = desert_main.iloc[y,x]
        except:
            risk = 0
    elif mapname == 'Baltic_Main':
        try:
            risk = baltic_main.iloc[y,x] 
        except:
            risk = 0
    elif mapname == 'Savage_Main':
        try:
            risk = savage_main.iloc[y,x] 
        except:
            risk = 0
    elif mapname == 'Summerland_Main':
        try:
            risk = summerland_main.iloc[y,x]
        except:
            risk = 0
        # correction due to lower max number of players
        risk = risk * 1,5625
    lst.append(risk)

# <font color=red> Preparing DataFrame </font>

In [ ]:
import os
import glob

path = 'telemetry_data/2020-04-09/pc'
for filename in glob.glob(os.path.join(path, '*.json')): #only process .JSON files in folder.      
    with open(filename, mode='r') as file:
        json_data = file.read()
        file.close()
    data = []
    head = []
    for i in json.loads(json_data):
        if i["_T"] in ("LogPlayerPosition", "LogParachuteLanding"):
            data.append(i)
        if i["_T"] in ("LogMatchStart"):
            head.append(i)
    

    # get events data into pandas dataframe
    data = pd.read_json(json.dumps(data))
    head = pd.read_json(json.dumps(head))
    
    # get map name
    mn = head.loc[0, 'mapName']
    
    # copy match id to all rows and drop LogMatchDefinition event
    # we will use this later to add use it for future merging tasks
    matchid = json.loads(json_data)[0]["MatchId"]
    data['MatchId'] = matchid

    #reset indexes
    data.reset_index(drop=True, inplace=True)

    # get player information
    from pandas.io.json import json_normalize
    
    # we need to expand the json documents on the character column
    data = pd.concat([data, json_normalize(data['character'])], axis=1).drop(['character', 'zone'], axis=1)
    
    def isGame(x):
        return x['isGame']

    data['isGame'] = data['common'].apply(isGame)
    data = data.drop('common', axis=1)

    # we will use the vehicle to check what players are still in the airplane
    data.vehicle = data.vehicle.apply(lambda x: {} if pd.isna(x) else x)
    data['vehicle'] = json_normalize(data.vehicle)['vehicleType']

    # check how many players' locations we have at each point in time
    # we want to make sure that after this process, all players have harmonized times and locations
    #for i in data.elapsedTime.unique():
        #obs = len(data[data['elapsedTime'] == i])
        #print(f'{i} has {obs} observations')

    # we create a new dataframe with the necessary information to interpolate locations
    data_new_coords = data[['elapsedTime','name','location.x','location.y', 'vehicle']].sort_values(['name','elapsedTime']).drop_duplicates()
    data_new_coords = data_new_coords.reset_index().drop('index', axis=1)
    data_new_coords = data_new_coords.dropna(subset = ['elapsedTime'])
    
    # we round coordinates for ease of calculations, and because a difference in location smaller than a cm is negligible
    data_new_coords['location.x'] = data_new_coords['location.x'].apply(round)
    data_new_coords['location.y'] = data_new_coords['location.y'].apply(round)

    data_new_coords.elapsedTime.unique().astype(int)
    
    # since different elapsed times don't show the location of all players, we want to interpolate the position of the players at each
    # point in time. e.g. if player a is in position (x,y) at second 10.0 and in position (x+10,y+10) at second 20.0
    # we can expect that this player will be at position (x+1,y+1) at second 11.0, and so on
    def getNewCoord(t, n):
        aa = data_new_coords[(data_new_coords['elapsedTime'] < t) & (data_new_coords['name'] == n)] # get previous observations
        bb = data_new_coords[(data_new_coords['elapsedTime'] > t) & (data_new_coords['name'] == n)] # get following observations
        if (bool(aa['location.x'].any())) & (bool(bb['location.x'].any())): # if we have both previous and following observation
            a = aa.iloc[-1] # get last previous observation
            b = bb.iloc[0] # get first following observation
            time = b[0] - a[0] # calculate the time difference
            # get x and y coordinate by calculating movement speed in cm/s and get full coordinate
            new_coord_x = a[2] + ((b[2] - a[2]) / time)*(t-a[0])  
            new_coord_y = a[3] + ((b[3] - a[3]) / time)*(t-a[0])
            # we also get the vehicle to check if a player has jumped from the airplane or not
            in_aircraft = b['vehicle']
            return new_coord_x, new_coord_y, in_aircraft
        elif (not bool(aa['location.x'].any())) & (bool(bb['location.x'].any())):
            # if there is no previous observation, set first following location to be current location
            in_aircraft = b['vehicle']
            return b.iat[0,2], b.iat[0,3], in_aircraft

        # If a player doesn't have a future location, that indicates the player is dead
        elif (bool(aa['location.x'].any())) & (not bool(bb['location.x'].any())):
            # if there is no following observation, we assume the player has died (we hope that only in the match)
            return 'dead', 'dead', 'dead'

    names=[]
    times=[]
    x_coords=[]
    y_coords=[]
    aircraft=[]
    # we will get the location every 10 seconds from second 1 (e.g. 1.0, 11.0, 21.0), and at the last second
    times_list = data_new_coords.elapsedTime.unique().astype(int)
    # we get the times in intervals of 10s from second 1 to the last second of the game
    unique_times_10s = np.arange(1, max(times_list), 10).tolist()
    # we also append the last second, regardless of the 10s interval
    unique_times_10s.append(max(times_list))
    for t in unique_times_10s:
        for n in data_new_coords.name.unique():
            # for every player, we get their location at these times
            a = data_new_coords[(data_new_coords['elapsedTime'] == t) & (data_new_coords['name'] == n)]
            # if the player has a location, we take it
            if a['location.x'].any():
                names.append(n)
                times.append(t)
                x_coords.append(a.iat[0,2])
                y_coords.append(a.iat[0,3])
                aircraft.append(a['vehicle'])
            # if not, we interpolate it
            else:
                x_coord, y_coord, in_aircraft = getNewCoord(t,n)
                names.append(n)
                times.append(t)
                x_coords.append(x_coord)
                y_coords.append(y_coord)
                aircraft.append(in_aircraft)

    # we create the dataframe from these locations
    players_location = pd.DataFrame(list(zip(times,names,x_coords, y_coords, aircraft)),
                 columns=['time','name','location.x','location.y', 'inAircraft'])
    # we also check if the players are or aren't in the aircraft with a binary variable
    players_location['inAircraft'] = players_location['inAircraft'].apply(lambda x: 1 if str(x) == 'TransportAircraft' else 0)


    # we make sure that harmonization of times and observations has been done
    #for i in players_location.time.unique():
        #obs = len(players_location[players_location['time'] == i])
        #print(f'{i} has {obs} observations')

    # encode player names
    # we do this to get the teamIds in the main dataframe
    players = data[['name', 'accountId','teamId']].drop_duplicates()
    players = players.sort_values('teamId').reset_index().drop(['index'], axis=1)


    # we merge these names to get accountId and teamId in the dataframe
    players_location = pd.merge(players_location,
                                players,
                                left_on = 'name',
                                right_on = 'name',
                                how = 'left')
    # moreover, since we are goint to later "asign" a number to the players (e.g. player1, player2), we want to make sure
    # that the criteria is consistent for all teams. Therefore, we do it with the .sort_values method
    # players on each team are ordered with this method inside the team
    players_location = players_location[players_location['time'] >= 0].sort_values(['teamId','time', 'name']).reset_index().drop(['index'], axis=1)

    players_location = players_location.replace('dead',0)

    # convert to 10 meters from cms
    players_location[['location.x','location.y']] = players_location[['location.x','location.y']].apply(lambda x: x.astype(int))
    players_location[['location.x','location.y']] = players_location[['location.x','location.y']].apply(lambda x: x // 1000)

    data = players_location[['location.x','location.y']]

    # convert df to list of tuples
    records = data.to_records(index=False)
    result = list(records)

    cols = ['risk']
    lst = []

    try:
        for i in result:
            risklevel(*i, mapname=mn)

        riskordered = pd.DataFrame(lst, columns=cols)
    
        # export match telemetry data into .csv format
        riskordered.to_csv(path_or_buf=f'{matchid}.csv')
    except:
        continue

In [37]:
pd.concat([players_location, riskordered], axis=1, sort=False)

,time,name,location.x,location.y,inAircraft,accountId,teamId,risk
0,1,Asin_Sira,254,948,0,account.154d4fd125a348e9a1f8df2daaabd9cb,1,0
1,1,Mac_1205,254,950,0,account.aae7ef8724284447953034a9ccbe89aa,1,0
2,1,dxnub,443,687,1,account.f41a6e2001664439b818784d15ffc124,1,0
3,1,spekture,254,952,0,account.6b46d60583254839bf6e8c3df814f115,1,0
4,11,Asin_Sira,258,805,0,account.154d4fd125a348e9a1f8df2daaabd9cb,1,0
5,11,Mac_1205,258,807,0,account.aae7ef8724284447953034a9ccbe89aa,1,0
6,11,dxnub,258,812,1,account.f41a6e2001664439b818784d15ffc124,1,0
7,11,spekture,258,808,0,account.6b46d60583254839bf6e8c3df814f115,1,0
8,21,Asin_Sira,262,664,0,account.154d4fd125a348e9a1f8df2daaabd9cb,1,0
9,21,Mac_1205,262,666,0,account.aae7ef8724284447953034a9ccbe89aa,1,0


## Export to .csv

In [30]:
# export match telemetry data into .csv format
players_location.to_csv(path_or_buf=f'{matchid}.csv')